In [164]:
f = open("cpu_list.txt","r")   # 读取cpu列表
lines = f.readlines()          # 读取全部内容 ，并以列表方式返回
length = len(lines)            # 行数
print("CPU数/（行数）：" + str(length))

CPU数/（行数）：37


In [165]:
lines

['Intel+Atom+D2550\n',
 'Intel+Atom+N2600\n',
 'Intel+Pentium+N3700\n',
 'Intel+Celeron+N3150\n',
 'Intel+Celeron+J3060\n',
 'Intel+Celeron+N2830\n',
 'Intel+Celeron+N2840\n',
 'Intel+Celeron+N2930\n',
 'Intel+Celeron+2955U\n',
 'Intel+Pentium+2117U\n',
 'Intel+Celeron+N3050\n',
 'Intel+Celeron+N3150\n',
 'Intel+Celeron+1037U\n',
 'Intel+Celeron+1017U\n',
 'Intel+Celeron+J3060\n',
 'Intel+Celeron+J3160\n',
 'Intel+Celeron+J3455\n',
 'Intel+Celeron+N3450\n',
 'Intel+Pentium+N3520\n',
 'Intel+Celeron+J1800\n',
 'Intel+Celeron+J1900\n',
 'Intel+Pentium+J2900\n',
 'Intel+Pentium+J3710\n',
 'Intel+Pentium+N3710\n',
 'Intel+Celeron+3865U\n',
 'Intel+Celeron+1037U\n',
 'Intel+Core+i3-3217UE\n',
 'Intel+Core+i3-3217U\n',
 'Intel+Core+i3-4005U\n',
 'Intel+Core+i5-4200Y\n',
 'Intel+Core+i5-4200U\n',
 'Intel+Core+i5-2450M\n',
 'Intel+Core+i5-5200U\n',
 'Intel+Core+i5-3427U\n',
 'Intel+Core+i5-2540M\n',
 'Intel+Core+i5-6360U\n',
 'Intel+Core+i7-7500U']

In [166]:
import requests
from bs4 import BeautifulSoup
import pprint
import json

def download_all_htmls():
    """
    下载所有列表页面的HTML，用于后续的分析
    """
    htmls = []
    for idx in lines:
        idx = idx.strip('\n')  # 去除txt文件读取后每行的"\n"
        url = f"https://www.cpubenchmark.net/cpu.php?cpu={idx}" # CPU的PassMark页面链接
        print("craw html:", url)
        r = requests.get(url)
        #if r.status_code != 200:
        #    raise Exception("error")
        htmls.append(r.text)
    return htmls

In [167]:
# 执行爬取
htmls = download_all_htmls()

craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Atom+D2550
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Atom+N2600
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Pentium+N3700
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Celeron+N3150
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Celeron+J3060
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Celeron+N2830
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Celeron+N2840
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Celeron+N2930
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Celeron+2955U
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Pentium+2117U
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Celeron+N3050
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Celeron+N3150
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Celeron+1037U
craw html: https://www.cpubenchmark.net/cpu.php?cpu=Intel+Celeron+1017

SSLError: HTTPSConnectionPool(host='www.cpubenchmark.net', port=443): Max retries exceeded with url: /cpu.php?cpu=Intel+Pentium+N3520 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:777)'),))

In [168]:
htmls[0]

'<!DOCTYPE html><html lang="en">\n<!-- CPU Becnhmark data Copyrighted by Passmark Software. Do not copy these charts or the information they contain -->\n<head>\n<title>PassMark - Intel Atom D2550 @ 1.86GHz - Price performance comparison</title>\n<meta charset="UTF-8">\n<meta name="DESCRIPTION" content="Performance and price comparison graphs for Intel Atom D2550 @ 1.86GHz">\n<meta name="KEYWORDS" content="benchmark, comparison, CPU, Intel Atom D2550 @ 1.86GHz, Atom-D2550">\n<link rel="stylesheet" href="/passmark.css?v=1.5" type="text/css">\n<link rel="stylesheet" href="/tabnav.css?v=1.5" type="text/css">\n<!-- <link rel="stylesheet" href="passmark_specials.css"> -->\n<link rel="stylesheet" href="/chart.css?v=1.5" type="text/css">\n<link rel="stylesheet" href="/cmp.css" type="text/css">\n<link rel="stylesheet" href="/dropdown.css" type="text/css">\n<link rel="stylesheet" href="/source/css/bootstrap.min.css" type="text/css">\n<meta name="viewport" content="width=device-width, initial-sc

In [169]:
def parse_single_html(html):
    """
    解析单个HTML，得到数据
    @return list({"link", "title", [label]})
    """
    soup = BeautifulSoup(html, 'html.parser')
    cpu_info = (
        soup.find_all("div", class_="desc")
    )
    datas = []
    for info in cpu_info:
        # 查找超链接
        
        # cpu名称
        cpu_name = info.find("div", class_="desc-body").find("div", class_="desc-header").find("span", class_="cpuname").get_text()
        
        # cpu发布时间、参数
        cpu_tdp = info.find("div", class_="desc-body").find("div", class_="left-desc-cpu").get_text()
        
        # cpu发布日、核心数等
        cpu_date = info.find("div", class_="desc-foot").find("p", class_="alt").get_text()
        
        # cpu分数等信息
        rating = info.find_all("div", class_="right-desc")
        cpu_mark = rating[0].get_text()
        
        datas.append(cpu_name)
        datas.append(cpu_tdp.replace("  W", "W") +  "  " + cpu_date.replace("\xa0\xa0", " "))
        datas.append(
            cpu_mark.replace(" Average CPU Mark\n ", "Average CPU Mark: ").replace("\n Single Thread Rating: ", "  Single Thread Rating: ").replace("\n     Samples: ", "  Samples: ").replace("\n      *Margin for error: ", "  *Margin for error: ").replace("\n+ Compare ", "").replace("\n", "")
        )
    
    # 当CPU名称错误时datas里加入错误CPU的名字
    if datas == []:
        cpu_info_error =  (soup.find("div", class_="productheader"))
        cpu_name_error = cpu_info_error.find("h1").get_text()
        datas.append("ErrorWarning: CPU name is wrong!!! " + cpu_name_error)
        
    return datas


In [170]:
pprint.pprint(parse_single_html(htmls[1]))

['Intel Atom N2600 @ 1.60GHz',
 'Class:  Laptop Socket: BGA559 Clockspeed: 1.6 GHz No of Cores: 2 (2 logical '
 'cores per physical) Typical TDP: 3.5W  CPU First Seen on Charts: Q1 2012',
 'Average CPU Mark: 531  Single Thread Rating: 269  Samples: 136*  *Margin for '
 'error: Low']


In [171]:
# 执行所有的HTML页面的解析
all_datas = []
for html in htmls:
    all_datas.extend(parse_single_html(html))
    len(all_datas)  

In [172]:
all_datas

['Intel Atom D2550 @ 1.86GHz',
 'Class:  Desktop Socket: BGA559 Clockspeed: 1.9 GHz No of Cores: 2 (2 logical cores per physical) Typical TDP: 10W  CPU First Seen on Charts: Q2 2012',
 'Average CPU Mark: 737  Single Thread Rating: 326  Samples: 59*  *Margin for error: Low',
 'Intel Atom N2600 @ 1.60GHz',
 'Class:  Laptop Socket: BGA559 Clockspeed: 1.6 GHz No of Cores: 2 (2 logical cores per physical) Typical TDP: 3.5W  CPU First Seen on Charts: Q1 2012',
 'Average CPU Mark: 531  Single Thread Rating: 269  Samples: 136*  *Margin for error: Low',
 'Intel Pentium N3700 @ 1.60GHz',
 'Class:  Laptop Socket: FCBGA1170 Clockspeed: 1.6 GHz Turbo Speed: 2.4 GHz No of Cores: 4 Typical TDP: 6W  CPU First Seen on Charts: Q2 2015',
 'Average CPU Mark: 1314  Single Thread Rating: 668 Cross-Platform Rating: 2495  Samples: 630*  *Margin for error: Low',
 'Intel Celeron N3150 @ 1.60GHz',
 'Class:  Laptop Socket: FCBGA1170 Clockspeed: 1.6 GHz Turbo Speed: 2.1 GHz No of Cores: 4 Typical TDP: 6W  CPU Firs

In [173]:
len(all_datas) # 输出行数

111

In [174]:
with open("cpu_info-multiple.json", "w") as fout:
    for data in all_datas:
        fout.write(json.dumps(data, ensure_ascii=False)+"\n")